Proyecto Clasificacion Binaria - Carlos Garcia 21000475

### Librerias

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn import svm, tree
from sklearn import metrics
from joblib import dump, load

In [3]:
if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()
  print("Enabled compatitility to tf1.x")

Instructions for updating:
non-resource variables are not supported in the long term
Enabled compatitility to tf1.x


### Obtencion de datos

In [4]:
data = pd.read_csv('./input/data_titanic_proyecto.csv')
data.head()

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,passenger_survived
0,1,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,Lower,M,N
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,Upper,F,Y
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Lower,F,Y
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,Upper,F,Y
4,5,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,Lower,M,N


In [5]:
#Train test split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, :-1], data[['passenger_survived']], test_size = 0.2, random_state = 123)

In [6]:
print('Original data: ', data.shape)
print('Xtrain: ', X_train.shape)
print('Xtest: ', X_test.shape)
print('Ytrain: ', y_train.shape)
print('Ytest: ', y_test.shape)

Original data:  (891, 12)
Xtrain:  (712, 11)
Xtest:  (179, 11)
Ytrain:  (712, 1)
Ytest:  (179, 1)


### Preparación de los datos

In [7]:
X_train.head()

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex
329,330,"Hippach, Miss. Jean Gertrude",16.0,0,1,111361,57.9792,B18,C,Upper,F
749,750,"Connaghton, Mr. Michael",31.0,0,0,335097,7.7500,NaN,Q,Lower,M
203,204,"Youseff, Mr. Gerious",45.5,0,0,2628,7.2250,NaN,C,Lower,M
421,422,"Charters, Mr. David",21.0,0,0,A/5. 13032,7.7333,NaN,Q,Lower,M
97,98,"Greenfield, Mr. William Bertram",23.0,0,1,PC 17759,63.3583,D10 D12,C,Upper,M


De todas las columnas disponibles Age, Cabin y Embarked presentan valores nulos. Dado que en las primeras dos columnas mencionadas el porcentaje es mayor a un 5% no es recomendable realizar imputación de los datos, por lo cual serán removidas del dataset. 

In [8]:
X_train.merge(y_train, left_index = True, right_index = True).isna().sum() / len(X_train)

PassengerId           0.000000
Name                  0.000000
Age                   0.207865
SibSp                 0.000000
Parch                 0.000000
Ticket                0.000000
Fare                  0.000000
Cabin                 0.780899
Embarked              0.002809
passenger_class       0.000000
passenger_sex         0.000000
passenger_survived    0.000000
dtype: float64

In [9]:
X_train.dropna(axis = 1, how = 'any', inplace = True)

La codificación de variables categóricas determina las pocas clases presentes tanto para passenger_sex por lo que se usara OHE k-1, en el caso de passenger_class al ser una variable ordinal se aplicara ordinal encoding, lo cual evita la expansión del número de features y toma ventaja del orden mostrado en la variable. 

In [10]:
enc = OrdinalEncoder()
pclass = np.sort(X_train.passenger_class.unique())
pclass = np.expand_dims(pclass, axis = 1)
enc.fit(pclass)

OrdinalEncoder()

In [11]:
X_train['passenger_class'] = enc.transform(X_train[['passenger_class']])
X_train.head()

,PassengerId,Name,SibSp,Parch,Ticket,Fare,passenger_class,passenger_sex
329,330,"Hippach, Miss. Jean Gertrude",0,1,111361,57.9792,2.0,F
749,750,"Connaghton, Mr. Michael",0,0,335097,7.7500,0.0,M
203,204,"Youseff, Mr. Gerious",0,0,2628,7.2250,0.0,M
421,422,"Charters, Mr. David",0,0,A/5. 13032,7.7333,0.0,M
97,98,"Greenfield, Mr. William Bertram",0,1,PC 17759,63.3583,2.0,M


In [12]:
X_train['passenger_sex'].unique()

array(['F', 'M'], dtype=object)

In [13]:
X_train['isFemale'] = np.where(X_train['passenger_sex'] == 'F', 1, 0)

In [14]:
X_train.drop(['PassengerId', 'Name', 'Ticket', 'passenger_sex'], axis = 1, inplace = True)
X_train.head()

,SibSp,Parch,Fare,passenger_class,isFemale
329,0,1,57.9792,2.0,1
749,0,0,7.7500,0.0,0
203,0,0,7.2250,0.0,0
421,0,0,7.7333,0.0,0
97,0,1,63.3583,2.0,0


In [15]:
y_train['passenger_survived'] = np.where(y_train['passenger_survived'] == 'Y', 1, 0)
y_train.head()

,passenger_survived
329,1
749,0
203,0
421,0
97,1


Finalmente se realiza el escalado de las variables para mantenerlas entre 0 y 1, con el fin de facilitar a los algoritmos de GD a converger

In [16]:
scaler = MinMaxScaler()
X_train.iloc[:, :-1] = scaler.fit_transform(X_train.iloc[:, :-1].values)
X_train.head()

,SibSp,Parch,Fare,passenger_class,isFemale
329,0.0,0.2,0.113168,1.0,1
749,0.0,0.0,0.015127,0.0,0
203,0.0,0.0,0.014102,0.0,0
421,0.0,0.0,0.015094,0.0,0
97,0.0,0.2,0.123667,1.0,0


In [17]:
#Train validation split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 123)

In [18]:
print('Xtrain: ', X_train.shape)
print('Xval: ', X_val.shape)
print('Ytrain: ', y_train.shape)
print('Yval: ', y_val.shape)

Xtrain:  (569, 5)
Xval:  (143, 5)
Ytrain:  (569, 1)
Yval:  (143, 1)


### Funciones Auxiliares

In [40]:
def getMetrics(config_str, y, yhat):
    model_metrics = dict()
    model_metrics['model'] = config_str
    model_metrics['accuracy'] = np.round(metrics.accuracy_score(y, yhat), 4)
    model_metrics['error'] = 1 - model_metrics['accuracy']
    model_metrics['precision'] = np.round(metrics.precision_score(y, yhat), 4)
    model_metrics['recall'] = np.round(metrics.recall_score(y, yhat), 4)
    model_metrics['f1-score'] = np.round(metrics.f1_score(y, yhat), 4)

    return pd.DataFrame(data = model_metrics, index = [0])

In [42]:
def writeModelResults(results, model, save_type = 'sklearn'):
    
    results.to_csv('./results/model_results.csv', mode = 'a', header = False, index = False)
    if save_type == 'sklearn':
        model_path = './models/' + results['model'].values[0] + '.joblib'
        dump(model, model_path) 
    else:
        pass

    print('Model results saved succesfully')
    return 

### Definición de Modelos

In [21]:
def train_decisionTree(x, y, depth = None):
    
    #Define model training
    clf = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = depth)
    clf = clf.fit(x.values, y.values)

    #Define configuration string
    features = '_'.join(list(x.columns))
    config_str = 'decisionTree_criterion=entropy__' + features
    
    return clf, config_str
    #tree.plot_tree(clf)

In [56]:
def train_SVM(x, y, c = 1, kernel = 'rbf'):
    
    #Define model training
    clf = svm.SVC()
    clf = clf.fit(x.values, np.squeeze(y.values, axis = 1))

    #Define configuration string
    features = '_'.join(list(x.columns))
    config_str = 'SVM_C={}_kernel={}__'.format(c, kernel) + features
    
    return clf, config_str


### Predicciones

In [57]:
model, config_str = train_SVM(X_train, y_train)

In [58]:
y_pred = model.predict(X_val)
y_pred

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0])

In [59]:
model_metrics = getMetrics(config_str, y_val.values, y_pred)
model_metrics

,model,accuracy,error,precision,recall,f1-score
0,SVM_C=1_kernel=rbf__SibSp_Parch_Fare_passenger...,0.8322,0.1678,0.8222,0.6981,0.7551


In [60]:
model_metrics['model'].values[0]

'SVM_C=1_kernel=rbf__SibSp_Parch_Fare_passenger_class_isFemale'

In [61]:
writeModelResults(model_metrics, model, save_type = 'sklearn')

Model results saved succesfully
